# Import libraries

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import sklearn

# Read CSV file

In [ ]:
df = pd.read_csv('../input/nyse/prices-split-adjusted.csv', na_values=['no info','0'])
print(df.head(5),'\n')
print(df.info(),'\n')

In [ ]:
df.describe()

# Ploting Data

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1,2,1)
plt.plot(df[df.symbol == 'AAPL'].close.values, color='red', label='close')
plt.plot(df[df.symbol == 'AAPL'].open.values, color='green', label='open')
plt.plot(df[df.symbol == 'AAPL'].high.values, color='yellow', label='high')
plt.plot(df[df.symbol == 'AAPL'].low.values, color='blue', label='low')
plt.title('stock price')
plt.xlabel('time [days]')
plt.ylabel('price')
plt.legend(loc='best')
#######################################################
plt.subplot(1,2,2)
plt.plot(df[df.symbol == 'AAPL'].volume.values, color='black', label='volume')
plt.title('stock volume')
plt.xlabel('time [days]')
plt.ylabel('volume')
plt.legend(loc='best')

# Normalizer and genarator

In [ ]:
def normalize_data(df):
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    df['open'] = min_max_scaler.fit_transform(df.open.values.reshape(-1,1))
    df['high'] = min_max_scaler.fit_transform(df.high.values.reshape(-1,1))
    df['low'] = min_max_scaler.fit_transform(df.low.values.reshape(-1,1))
    df['close'] = min_max_scaler.fit_transform(df['close'].values.reshape(-1,1))
    df['volume'] = min_max_scaler.fit_transform(df.volume.values.reshape(-1,1))

    return df

In [ ]:
def generator(dataframe, batch_size, input_timestep, output_timestep):
    x_train = []
    y_train = []
    full_size = np.array(dataframe).shape[0]
    counter = 0
    index = 0
    while index <= (full_size - input_timestep - output_timestep):
        
        if batch_size == counter:
            x_train_1 = np.reshape(x_train,(batch_size,input_timestep,-1 ))
            y_train_2 = np.reshape(y_train,(batch_size,output_timestep,-1))
            x_train=[]
            y_train=[]
            counter = 0
            yield np.array(x_train_1), np.array(y_train_2)
            
        
        x_train.append(np.array(dataframe.iloc[index:input_timestep + index]).reshape(input_timestep, -1))
        y_train.append(np.array(dataframe.iloc[input_timestep + index:+input_timestep+output_timestep+index]).reshape(output_timestep, -1))
        counter += 1
        index += 1
        

# Model

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.LSTM(20, return_sequences=True, input_shape=[10,5], name="LSTM1"),
        tf.keras.layers.LSTM(20, return_sequences=True,name="LSTM2"),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5), name="Dense")
    ]
)

In [ ]:
model.summary()

# Prepairing data and training

In [ ]:
df = normalize_data(df)
df_total = df[['open','close','high','low','volume']][df.symbol == 'AAPL']
df_train = df[['open','close','high','low','volume']][df.symbol == 'AAPL'].iloc[0:1752]
df_train = normalize_data(df_train)
df_test = df[['open','close','high','low','volume']][df.symbol == 'AAPL'].iloc[1752:1762]
df_test = normalize_data(df_test)

In [ ]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

In [ ]:
def loss(model,true,inputs):
    with tf.GradientTape() as tape:
        loss_value = tf.keras.losses.MSE(true, model(inputs))
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
def train_Model(model,epoch,gen):
    print(model.summary(),"\n")
    print("Training Model..... :  ","\n")
    for i in range(0,epoch):
        print("EPOCH "+str(i)+" :  ","\n")
        for x , y in gen:
            print(x.shape)
            print(y.shape)
            loss_value, grads = loss(model,y, x)
            print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(), loss_value.numpy()))
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return model

In [ ]:
model = train_Model(model,1,generator(df_train,32,10,10))

In [ ]:
x_test = np.array(df_test)
print(x_test.shape)
x_test = np.reshape(x_test,[1,10,5])
out = model.predict(x_test)
new_data = np.concatenate((np.array(df_train),out.reshape(10,5)),axis=0)
old_data = np.array(df_total)
print(new_data.shape)
print(old_data.shape)


In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(old_data[1752:,0], color='red', label='close')
plt.plot(old_data[1752:,1], color='green', label='open')
plt.plot(old_data[1752:,2], color='yellow', label='high')
plt.plot(old_data[1752:,3], color='blue', label='low')
plt.title('stock price')
plt.xlabel('time [days]')
plt.ylabel('price')
plt.legend(loc='best')

plt.subplot(1,2,2)
plt.plot(new_data[1752:,0], color='red', label='close')
plt.plot(new_data[1752:,1], color='green', label='open')
plt.plot(new_data[1752:,2], color='yellow', label='high')
plt.plot(new_data[1752:,3], color='blue', label='low')
plt.title('stock price')
plt.xlabel('time [days]')
plt.ylabel('price')
plt.legend(loc='best')